In [ ]:
pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=d3356634f49d4aeadb5002fe6dea8fb4ff647f9c4829e5f836f7e58a67342e15
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [ ]:
import pandas as pd
import pandasql as psql

In [ ]:
orders = pd.read_csv('orders.csv_orders.csv.csv')
order_details = pd.read_csv('order_details.csv_order_details.csv.csv')
pizza_types = pd.read_csv('pizza_types.csv_pizza_types.csv.csv', encoding='ISO-8859-1')
pizzas = pd.read_csv('pizzas.csv_pizzas.csv.csv')

In [ ]:
# Q1: The total number of orders placed
query1 = """
SELECT COUNT(*) AS total_orders
FROM orders;
"""

# Execute the query
result1 = psql.sqldf(query1, locals())
print(result1)

   total_orders
0         21350


In [ ]:
# Q2: The total revenue generated from pizza sales
query2 = """
SELECT ROUND(SUM(p.price * od.quantity), 2) AS total_revenue
FROM order_details od
JOIN pizzas p ON od.pizza_id = p.pizza_id;
"""
# Execute the query
result2 = psql.sqldf(query2, locals())
print(result2)

   total_revenue
0      817860.05


In [ ]:
# Q3: The highest priced pizza
query3 = """
SELECT p.pizza_id, pt.name, p.size, p.price
FROM pizzas p
JOIN pizza_types pt ON p.pizza_type_id = pt.pizza_type_id
ORDER BY p.price DESC
LIMIT 1;
"""
# Execute the query
result3 = psql.sqldf(query3, locals())
print(result3)

        pizza_id             name size  price
0  the_greek_xxl  The Greek Pizza  XXL  35.95


In [ ]:
# Q4: The most common pizza size ordered
query4 = """
SELECT p.size, COUNT(*) AS order_count
FROM order_details od
JOIN pizzas p ON od.pizza_id = p.pizza_id
GROUP BY p.size
ORDER BY order_count DESC LIMIT 1;
"""
# Execute the query
result4 = psql.sqldf(query4, locals())
print(result4)

  size  order_count
0    L        18526


In [ ]:
# Q5: The top 5 most ordered pizza types along with their quantities
query5 = """
SELECT pt.name, SUM(od.quantity) AS total_quantity
FROM order_details od
JOIN pizzas p ON od.pizza_id = p.pizza_id
JOIN pizza_types pt ON p.pizza_type_id = pt.pizza_type_id
GROUP BY pt.name
ORDER BY total_quantity DESC
LIMIT 5;
"""
# Execute the query
result5 = psql.sqldf(query5, locals())
print(result5)

                         name  total_quantity
0    The Classic Deluxe Pizza            2453
1  The Barbecue Chicken Pizza            2432
2          The Hawaiian Pizza            2422
3         The Pepperoni Pizza            2418
4      The Thai Chicken Pizza            2371


In [ ]:
# Q6: The quantity of each pizza category ordered
query6 = """
SELECT pt.category, SUM(od.quantity) AS total_quantity
FROM order_details od
JOIN pizzas p ON od.pizza_id = p.pizza_id
JOIN pizza_types pt ON p.pizza_type_id = pt.pizza_type_id
GROUP BY pt.category
ORDER BY total_quantity DESC;
"""
# Execute the query
result6 = psql.sqldf(query6, locals())
print(result6)

  category  total_quantity
0  Classic           14888
1  Supreme           11987
2   Veggie           11649
3  Chicken           11050


In [ ]:
# Q7: The distribution of orders by hours of the day
query7 = """
SELECT SUBSTR(time, 1, 2) AS hour, COUNT(*) AS order_count
FROM orders
GROUP BY hour
ORDER BY hour;
"""
# Execute the query
result7 = psql.sqldf(query7, locals())
print(result7)

   hour  order_count
0    09            1
1    10            8
2    11         1231
3    12         2520
4    13         2455
5    14         1472
6    15         1468
7    16         1920
8    17         2336
9    18         2399
10   19         2009
11   20         1642
12   21         1198
13   22          663
14   23           28


In [ ]:
# Q8: The category-wise distribution of pizzas
query8 = """
SELECT pt.category, COUNT(*) AS pizza_count
FROM pizzas p
JOIN pizza_types pt ON p.pizza_type_id = pt.pizza_type_id
GROUP BY pt.category;
"""

# Execute the query
result8 = psql.sqldf(query8, locals())
print(result8)

  category  pizza_count
0  Chicken           18
1  Classic           26
2  Supreme           25
3   Veggie           27


In [ ]:
# Q9: The average number of pizzas ordered per day
query9 = """
SELECT ROUND(AVG(daily_orders), 2) AS avg_pizzas_per_day
FROM (
    SELECT date, COUNT(*) AS daily_orders
    FROM orders
    GROUP BY date
) AS daily;
"""

# Execute the query
result9 = psql.sqldf(query9, locals())
print(result9)

   avg_pizzas_per_day
0               59.64


In [ ]:
# Q10: Top 3 most ordered pizza types based on revenue
query10 = """
SELECT pt.name, ROUND(SUM(p.price * od.quantity), 2) AS total_revenue
FROM order_details od
JOIN pizzas p ON od.pizza_id = p.pizza_id
JOIN pizza_types pt ON p.pizza_type_id = pt.pizza_type_id
GROUP BY pt.name
ORDER BY total_revenue DESC
LIMIT 3;
"""

# Execute the query
result10 = psql.sqldf(query10, locals())
print(result10)

                           name  total_revenue
0        The Thai Chicken Pizza       43434.25
1    The Barbecue Chicken Pizza       42768.00
2  The California Chicken Pizza       41409.50


In [ ]:
# Q11: The percentage contribution of each pizza type to revenue
query11 = """
SELECT pt.name, ROUND((SUM(p.price * od.quantity) / total_revenue.total) * 100, 2) AS percentage_contribution
FROM order_details od
JOIN pizzas p ON od.pizza_id = p.pizza_id
JOIN pizza_types pt ON p.pizza_type_id = pt.pizza_type_id
JOIN (SELECT SUM(p.price * od.quantity) AS total FROM order_details od JOIN pizzas p ON od.pizza_id = p.pizza_id) AS total_revenue
GROUP BY pt.name
ORDER BY percentage_contribution DESC;
"""

# Execute the query
result11 = psql.sqldf(query11, locals())
print(result11)

                                          name  percentage_contribution
0                       The Thai Chicken Pizza                     5.31
1                   The Barbecue Chicken Pizza                     5.23
2                 The California Chicken Pizza                     5.06
3                     The Classic Deluxe Pizza                     4.67
4                      The Spicy Italian Pizza                     4.26
5                  The Southwest Chicken Pizza                     4.24
6                    The Italian Supreme Pizza                     4.09
7                           The Hawaiian Pizza                     3.95
8                        The Four Cheese Pizza                     3.95
9                           The Sicilian Pizza                     3.78
10                         The Pepperoni Pizza                     3.69
11                             The Greek Pizza                     3.48
12                          The Mexicana Pizza                  

In [ ]:
# Q12: The cumulative revenue generated over time
query12 = """
SELECT date, SUM(daily_revenue) OVER (ORDER BY date) AS cumulative_revenue
FROM (
    SELECT date, SUM(p.price * od.quantity) AS daily_revenue
    FROM orders o
    JOIN order_details od ON o.order_id = od.order_id
    JOIN pizzas p ON od.pizza_id = p.pizza_id
    GROUP BY date
) AS daily;
"""

# Execute the query
result12 = psql.sqldf(query12, locals())
print(result12)

           date  cumulative_revenue
0    2015-01-01             2713.85
1    2015-01-02             5445.75
2    2015-01-03             8108.15
3    2015-01-04             9863.60
4    2015-01-05            11929.55
..          ...                 ...
353  2015-12-27           810615.80
354  2015-12-28           812253.00
355  2015-12-29           813606.25
356  2015-12-30           814944.05
357  2015-12-31           817860.05

[358 rows x 2 columns]


In [ ]:
# Q13: The top 3 most ordered pizza types based on revenue for each pizza category
query13 = """
SELECT pt.category, pt.name, ROUND(SUM(p.price * od.quantity), 2) AS total_revenue
FROM order_details od
JOIN pizzas p ON od.pizza_id = p.pizza_id
JOIN pizza_types pt ON p.pizza_type_id = pt.pizza_type_id
GROUP BY pt.category, pt.name
ORDER BY pt.category, total_revenue DESC
LIMIT 3;
"""

# Execute the query
result13 = psql.sqldf(query13, locals())
print(result13)

  category                          name  total_revenue
0  Chicken        The Thai Chicken Pizza       43434.25
1  Chicken    The Barbecue Chicken Pizza       42768.00
2  Chicken  The California Chicken Pizza       41409.50
